In [1]:
import os
import torch
import transformers
import peft
import datasets
import evaluate
import time
assert torch.cuda.is_available(), "you need cuda for this part"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = 'hodza/BlackBox-Coder-3B'
base_model_name = 'Qwen/Qwen2.5-Coder-3B-Instruct'

merged_model_path = f'/app/models/BlackBox-Coder-3B-merged'

In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained(base_model_name)

In [4]:
base_model  = transformers.AutoModelForCausalLM.from_pretrained(
    base_model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

Loading checkpoint shards: 100%|██████████| 2/2 [02:35<00:00, 77.80s/it] 


In [5]:
model = peft.PeftModel.from_pretrained(base_model, model_name)

In [6]:
merged_model = model.merge_and_unload()

In [7]:

    
# Сохранение объединённой модели


merged_model.save_pretrained(merged_model_path)
tokenizer.save_pretrained(merged_model_path)

('/app/models/BlackBox-Coder-3B-merged/tokenizer_config.json',
 '/app/models/BlackBox-Coder-3B-merged/special_tokens_map.json',
 '/app/models/BlackBox-Coder-3B-merged/vocab.json',
 '/app/models/BlackBox-Coder-3B-merged/merges.txt',
 '/app/models/BlackBox-Coder-3B-merged/added_tokens.json',
 '/app/models/BlackBox-Coder-3B-merged/tokenizer.json')

In [8]:
system = '''
Ты помощник для работы в системе BlackBox с использованием языка Component Pascal. Твоя задача информативно отвечать на вопросы. 
Ответ необходимо предоставить в формате markdown и выделять код символами ```. 

Пример: 

**Вопрос**: Как реализовать сортировку пузырьком?
**Ответ**: 

Cортировку пузырьком можно реализовать с помощью следующего кода:

```
	PROCEDURE BubbleSort*;  
		VAR i, j: INTEGER; x: Item;
	BEGIN
		FOR i := 1 TO n-1 DO
			FOR j := n-1 TO i BY -1 DO
				IF a[j-1] > a[j] THEN
					x := a[j-1];  a[j-1] := a[j];  a[j] := x
				END
			END
		END
	END BubbleSort;
```
'''

In [9]:
def prompt_to_chat(prompt, system = None):
    messages = [
        {"role": "user", "content": prompt}
    ]
    if system is not None:
        messages.insert(0, {"role": "system", "content": prompt})
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return text

def infer(model, prompt, l=100, use_chat = True, temperature=0.4, top_p = 0.8, system = None):
    if use_chat:
        prompt = prompt_to_chat(prompt, system)
    model_inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=l,
        #temperature=temperature, 
        #top_p=top_p,
        do_sample=True ,  
        repetition_penalty = 1.1,
        temperature = 0.1,
        top_p = 0.3,
        top_k = 20,
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=False)[0]
    return response

In [10]:
print(infer(merged_model, 'Как мне вывести массив в Log?', 512, use_chat = True, system = system))

Вообще-то, это не такая сложная задача. В качестве примера рассмотрим следующий код:

MODULE  i21примеры_ЗаписьМассива;

	IMPORT  StdLog,  Random := StdLibRandom;
	
	VAR  
		a: POINTER TO ARRAY OF INTEGER; (* указатель на массив целых чисел *)
		n: INTEGER; (* длина массива *)
		
BEGIN
	(* создаем массив из n элементов и заполняем его случайными числами *)
	n := 50;
	a := SYSTEM.Aloc( n );
	StdLog.String('создан массив из '); StdLog.Int( n ); StdLog.Ln;
	StdLog.String('заполнен массив случайными числами'); StdLog.Ln;
	StdLog.Ln;
	
	(* выводим содержимое массива в лог *)
END  i21примеры_ЗаписьМассива.

Чтобы посмотреть результат выполнения программы, нужно кликнуть по кнопке "Выполнить" (Run) или нажать Ctrl+F9.<|im_end|>
